In [1]:
from __future__ import division, print_function

#def get_gapped_kmer_embedding_filters_and_func(
#        kmer_len, alphabet, num_gaps, max_mismatches):
#    filters, biases, string_reps = prep_filters_and_biases(
#                                    kmer_len=kmer_len,
#                                    alphabet=alphabet,
#                                    num_gaps=num_gaps,
#                                    max_mismatches=max_mismatches) 
#    func = B.get_gapped_kmer_embedding_func(filters=filters, biases=biases)
#    return filters, string_reps, func


#This code can't handle mismatches, so mismatches are assumed to be 0
def get_sparse_gapped_kmer_embeddings_func(kmer_len, alphabet, num_gaps):
    pass #Fill me out!
    #You should return a function like get_sparse_representations,
    #defined below
    

In [2]:
import numpy as np

#Recap of general idea.

#Demo of filters in the case of 3-mers with 1 gap

#Step 1:
#First, we figure out all the possible non-redundant combinations of non-gapped positions. They are:
#XX*
#X*X
#NOT *XX as it is redundant with XX*

#Step 2:
#For each of the non-redundant combinations of non-gapped positions, we create a filter.
#The filter corresponding to XX* has
# 3 rows (corresponding to the fact that we are using 3-mers) and 4 columns (corresponding to ACGT)

XXstar_filter = np.array([
    [0,1,2,3],
    [0,4,8,12],
    [0,0,0,0]
])

#The row(s) corresponding to the gap(s) have all-zeros
#The rows corresponding to the bases have numbers.
#The place-value is determined by the index of the column
#Depending on the kmer that is scanned, all values from 0 to 15
#Can be output by this filter.
#AA* would output a 0, TT* would output a 15
XstarX_filter = np.array([
    [0,1,2,3],
    [0,0,0,0],
    [0,4,8,12]
])

#Scanning with this filter alone, A*A would output a 0, T*T would output a 15
#To distinguish between AA* and A*A, we will add 16 to the outputs of the XstarX filter.
#So A*A would output 16, and T*T would output 31.
#If we had had a third non-redundant filter, we would have added 32 to the output to
#maintain the distinctness.


In [3]:

#This function can be used to compile
#A tensorflow function
def compile_func(inputs, outputs):
    inputs = [inputs]
    def func_to_return(inp):
        inp = [inp]
        feed_dict = {}
        for input_tensor, input_val in zip(inputs, inp):
            feed_dict[input_tensor] = input_val 
        sess = get_session()
        return sess.run(outputs, feed_dict=feed_dict)  
    return func_to_return

def get_session():
    try:
        #use the keras session if there is one
        import keras.backend as K
        return K.get_session()
    except:
        #Warning: I haven't really tested this behaviour out...
        global _SESS 
        if _SESS is None:
            print("MAKING A SESSION")
            _SESS = tf.Session()
            _SESS.run(tf.global_variables_initializer()) 
        return _SESS

#Functions to do one-hot encoding from sequence
def one_hot_encode_along_channel_axis(sequence):
    #theano dim ordering, uses row axis for one-hot
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1


In [4]:
import tensorflow as tf
from collections import Counter

#create a tensor representing our inputs
#Our inputs are 3-dimensional; first axis is the example index,
#second axis is the length, third axis is ACGT

#The last axis of filters is the filter type, hence this line of code
#The type of the numpy array should be float32 to work with tensorflow
filters = np.concatenate([XXstar_filter[:,:,None], XstarX_filter[:,:,None]],axis=2).astype("float32")
print(filters.shape)

input_len = 10

input_onehot_tensor = tf.placeholder(shape=(None, input_len, 4),
                                     dtype=tf.float32,
                                     name="input_onehot_tensor")
conv_out = tf.nn.conv1d(
    value=input_onehot_tensor,
    filters=filters,
    stride=1,
    #valid padding means that when you scan the sequence, you don't go over the end
    padding="VALID")

#We add 0 to the output of the first channel and 16 to the output
#of the second channel
conv_out_plus_constants = conv_out + np.array([0,16])[None,None,:]
conv_out_flattened = tf.reshape(conv_out_plus_constants, (-1,(input_len-filters.shape[0]+1)
                                                             *filters.shape[-1]))

conv_func = compile_func(inputs=input_onehot_tensor,
                         outputs=conv_out_flattened)

#This function is basically like what you would have to return
#as the embedding function
def get_sparse_representations(input_onehot):
    gkmer_matches = np.array(conv_func(onehot_sequences)).astype("int32")
    return [Counter(x) for x in gkmer_matches]


Couldn't import dot_parser, loading of dot files will not be possible.
(3, 4, 2)


/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
example_sequences = ["AAAAAAAAAA", "TTTTTTTTTT", "AATAGCTGAT"]
onehot_sequences = np.array([one_hot_encode_along_channel_axis(x) for x in example_sequences])
print(onehot_sequences.shape)


(3, 10, 4)


In [6]:
get_sparse_representations(onehot_sequences)

Using TensorFlow backend.


[Counter({0: 8, 16: 8}),
 Counter({15: 8, 31: 8}),
 Counter({0: 1,
          2: 1,
          3: 1,
          6: 1,
          8: 1,
          11: 1,
          12: 1,
          13: 1,
          16: 1,
          19: 1,
          20: 1,
          25: 1,
          27: 1,
          28: 1,
          30: 2})]

In [7]:
#How to get the list of non-redundant non-zero positions in an automated way:

import itertools
from collections import OrderedDict

kmer_len = 3
num_gaps = 1
#given the number of gaps and the kmer length,
#find all unique combinations of positions
#in the filter that are nonzero (nonzero = not a gap)
nonzero_position_combos = list(itertools.combinations(
                        iterable=range(kmer_len),
                        r=(kmer_len-num_gaps)))
print("Redundant nonzero positions",nonzero_position_combos)

#You have to filter that list for redundancy
#To do so, create a string representation of them, and
#trim the string for gaps at the beginning and end

stripped_string_representation_to_nonzero_positions = OrderedDict()
for nonzero_positions in nonzero_position_combos:
    characters = [" " for x in range(kmer_len)]
    for nonzero_position in nonzero_positions:
        characters[nonzero_position] = "X"
    string_representation = "".join(characters)
    stripped_string_representation = string_representation.rstrip().lstrip()
    print(nonzero_positions, string_representation, stripped_string_representation)
    if stripped_string_representation in stripped_string_representation_to_nonzero_positions:
        print("Skipping",nonzero_positions,"as it's redundant with",
              stripped_string_representation_to_nonzero_positions[stripped_string_representation])
    else:
        stripped_string_representation_to_nonzero_positions[stripped_string_representation] = nonzero_positions

print("Non redundant nonzero positions",stripped_string_representation_to_nonzero_positions.values())

Redundant nonzero positions [(0, 1), (0, 2), (1, 2)]
(0, 1) XX  XX
(0, 2) X X X X
(1, 2)  XX XX
Skipping (1, 2) as it's redundant with (0, 1)
Non redundant nonzero positions [(0, 1), (0, 2)]
